# Step 1: Creating Tokens

In [1]:
# Reading the file and printing total number of characters
with open("dataset/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"The total number of characters: {len(raw_text)}")
# Printing the first 100 characters
print(raw_text[:99])

The total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
# Splitting the text into individual tokens

import re

sample_text = "Hello, world. This, is a test."
# spiltting wherever whitespace is encountered
result = re.split(r'(\s)', sample_text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [5]:
#splitting "." and "," as well
result = re.split(r'([,.]|\s)', sample_text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [6]:
# Removing whitespaces from the tokens
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [8]:
sample_text = "Hello, world. Is this-- a test?"
# Adding other special characters also to the token list
result = re.split(r'([,.:;?_!"()\']|--|\s)', sample_text)
# Removing whitespaces from the token list
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [11]:
# Applying the basic tokenizer to the raw text 
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30])
print(f"total number of tokens: {len(preprocessed)}")

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
total number of tokens: 4690


# Step 2: Creating Token IDs

In [13]:
# Assigning token IDs to each of the tokens
# Step 1: Creating list of unique tokens 
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(f"The size of the vocabulary is {vocab_size}")

The size of the vocabulary is 1130


In [14]:
## Creating the vocabulary
vocab = {token:integer for integer,token in enumerate(all_words)}

#Printing first 50 elements of the vocab
for i, item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


# Tokenizer Class

In [15]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])',r'\1', text)
        return text

In [18]:
# Testing the Tokenizer on the text already present in the training set (raw_text)
tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know,"
          Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(f"The Encoded Tokens are : {ids}")
print(f"The decoded Text is: {tokenizer.decode(ids)}")

The Encoded Tokens are : [56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
The decoded Text is: It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [20]:
# Now testing the tokenizer on the text that is not present in the training set (raw_text)
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

Here we can clearly see that for the text which is not present in the training set (raw_text) the encoder fails
"Hello" is not present in the vocabulary

# Adding Special Context Tokens

In [21]:
## Adding tokens "<|unk|>" and <|endoftext|>
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
print(f'The length of the vocab is: {len(vocab.items())}')

The length of the vocab is: 1132


In [22]:
#printing the last 5 entries of the updated vocab
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [23]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])',r'\1', text)
        return text

In [27]:
# Testing the SimpleTokenizerV2
tokenizer = SimpleTokenizerV2(vocab)
text_1 = "Hello, do you like tea?"
text_2 = "In the sunlit terraces of the palace."

# joining text_1 and text_2 by Adding the <|endoftext|> token in between text_1 and text_2
text = " <|endoftext|> ".join((text_1, text_2))
print(f"The preprocessed text is: {text}")
ids = tokenizer.encode(text)
print(f"The encoded text is: {ids}")
print(f"The decoded text is: {tokenizer.decode(ids)}")

The preprocessed text is: Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
The encoded text is: [1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
The decoded text is: <|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.
